In [108]:
#import things
import numpy as np
from pandas import DataFrame as df
import itertools, json, os, re
import seaborn as sns
import matplotlib.pyplot as plt


In [109]:
#define lexicons and visuals

def get_cleanup_elements():
    objects = ['orange_spoon', 'pink_rug', 'blue_teapot', 'yellow_lamp']
    rooms = ['red_hall', 'green_kitchen', 'purple_bedroom']
    return rooms, objects

def get_shapeworld_elements():
    objects = sorted(['red_square', 'green_square', 'red_circle', 'green_circle'])
    attributes = sorted(['square', 'circle', 'green', 'red', 'red_square', 'red_circle', 'green_square', 'green_circle'])
    return attributes, objects

def lexicon_true(attr, obj):
    attr_items, obj_items = attr.split('_'), obj.split('_')
    return len(list(set(attr_items) & set(obj_items))) >= len(attr_items)

def hard_coded_lexicon():
    #rooms, objects = get_cleanup_elements()
    rows, cols = get_shapeworld_elements()
    nrows, ncols = len(rows), len(cols)

    matrix = np.zeros((nrows, ncols))
    for i in range(len(rows)):
        for j in range(len(cols)):
            if lexicon_true(rows[i], cols[j]):
                matrix[i][j] = 1.0
                
    dataframe = df(data = matrix, columns = cols, index = rows)
    return dataframe

def add_prior(objects, prior, lex):
    rows, cols = lex.index, lex.columns
    matrix = lex.values

    for i in range(len(prior)):
        for row in range(len(matrix)):
            matrix[row][i] *= prior[i]

    print('Fine')
    dataframe = df(data = matrix, columns = cols, index = rows)
    print('Fine')
    matrix = row_norm(matrix)
    print('Fine')
    dataframe = df(data = matrix, columns = cols, index = rows)
    print('Fine')
    return dataframe

def visualise(df):
    print('Dataframe:\n', df)
    sns.set()
    refs, utterances = list(df.index), list(df.columns)
    print(refs, utterances)
    
    ax = sns.heatmap(df.values, cmap="YlGnBu")
    ax.set_xlabel('Referents'); ax.set_ylabel('Utterances')
    ax.set(yticklabels=refs)
    ax.set(xticklabels=utterances)
    plt.setp(ax.get_yticklabels(), rotation=45);
    plt.show()


In [110]:
# testing utils

df = hard_coded_lexicon()
#visualise(df)
df = add_prior([], [0.7, 0.1, 0.1, 0.1], df)
visualise(df)

Fine


TypeError: 'DataFrame' object is not callable

In [ ]:
#define modules for agents